# Part 3: Single-View Geometry

## Usage
This code snippet provides an overall code structure and some interactive plot interfaces for the *Single-View Geometry* section of Assignment 3. In [main function](#Main-function), we outline the required functionalities step by step. Some of the functions which involves interactive plots are already provided, but [the rest](#Your-implementation) are left for you to implement.

## Package installation
- In this code, we use `tkinter` package. Installation instruction can be found [here](https://anaconda.org/anaconda/tk).

# Common imports

In [18]:

%matplotlib tk
import matplotlib.pyplot as plt
import numpy as np

from PIL import Image

In [19]:
print("hello")

hello


# Provided functions

In [20]:
def get_input_lines(im, min_lines=3):
    """
    Allows user to input line segments; computes centers and directions.
    Inputs:
        im: np.ndarray of shape (height, width, 3)
        min_lines: minimum number of lines required
    Returns:
        n: number of lines from input
        lines: np.ndarray of shape (3, n)
            where each column denotes the parameters of the line equation
        centers: np.ndarray of shape (3, n)
            where each column denotes the homogeneous coordinates of the centers
    """
    n = 0
    lines = np.zeros((3, 0))
    centers = np.zeros((3, 0))

    plt.figure()
    plt.imshow(im)
    plt.show()
    print('Set at least %d lines to compute vanishing point' % min_lines)
    while True:
        print('Click the two endpoints, use the right key to undo, and use the middle key to stop input')
        clicked = plt.ginput(2, timeout=0, show_clicks=True)
        if not clicked or len(clicked) < 2:
            if n < min_lines:
                print('Need at least %d lines, you have %d now' % (min_lines, n))
                continue
            else:
                # Stop getting lines if number of lines is enough
                break

        # Unpack user inputs and save as homogeneous coordinates
        pt1 = np.array([clicked[0][0], clicked[0][1], 1])
        pt2 = np.array([clicked[1][0], clicked[1][1], 1])
        # Get line equation using cross product
        # Line equation: line[0] * x + line[1] * y + line[2] = 0
        line = np.cross(pt1, pt2)
        lines = np.append(lines, line.reshape((3, 1)), axis=1)
        # Get center coordinate of the line segment
        center = (pt1 + pt2) / 2
        centers = np.append(centers, center.reshape((3, 1)), axis=1)

        # Plot line segment
        plt.plot([pt1[0], pt2[0]], [pt1[1], pt2[1]], color='b')

        n += 1

    return n, lines, centers

In [4]:
def plot_lines_and_vp(im, lines, vp):
    """
    Plots user-input lines and the calculated vanishing point.
    Inputs:
        im: np.ndarray of shape (height, width, 3)
        lines: np.ndarray of shape (3, n)
            where each column denotes the parameters of the line equation
        vp: np.ndarray of shape (3, )
    """
    bx1 = min(1, vp[0] / vp[2]) - 10
    bx2 = max(im.shape[1], vp[0] / vp[2]) + 10
    by1 = min(1, vp[1] / vp[2]) - 10
    by2 = max(im.shape[0], vp[1] / vp[2]) + 10

    plt.figure()
    plt.imshow(im)
    for i in range(lines.shape[1]):
        if lines[0, i] < lines[1, i]:
            pt1 = np.cross(np.array([1, 0, -bx1]), lines[:, i])
            pt2 = np.cross(np.array([1, 0, -bx2]), lines[:, i])
        else:
            pt1 = np.cross(np.array([0, 1, -by1]), lines[:, i])
            pt2 = np.cross(np.array([0, 1, -by2]), lines[:, i])
        pt1 = pt1 / pt1[2]
        pt2 = pt2 / pt2[2]
        plt.plot([pt1[0], pt2[0]], [pt1[1], pt2[1]], 'g')

    plt.plot(vp[0] / vp[2], vp[1] / vp[2], 'ro')
    plt.show()

In [5]:
def get_top_and_bottom_coordinates(im, obj):
    """
    For a specific object, prompts user to record the top coordinate and the bottom coordinate in the image.
    Inputs:
        im: np.ndarray of shape (height, width, 3)
        obj: string, object name
    Returns:
        coord: np.ndarray of shape (3, 2)
            where coord[:, 0] is the homogeneous coordinate of the top of the object and coord[:, 1] is the homogeneous
            coordinate of the bottom
    """
    plt.figure()
    plt.imshow(im)

    print('Click on the top coordinate of %s' % obj)
    clicked = plt.ginput(1, timeout=0, show_clicks=True)
    x1, y1 = clicked[0]
    # Uncomment this line to enable a vertical line to help align the two coordinates
    # plt.plot([x1, x1], [0, im.shape[0]], 'b')
    print('Click on the bottom coordinate of %s' % obj)
    clicked = plt.ginput(1, timeout=0, show_clicks=True)
    x2, y2 = clicked[0]

    plt.plot([x1, x2], [y1, y2], 'b')

    return np.array([[x1, x2], [y1, y2], [1, 1]])

# Your implementation

In [6]:
import numpy as np
def get_vanishing_point(lines):
    """
    Solves for the vanishing point using the user-input lines.
    """
    n = lines.shape[1]
    num_intersects = n * (n-1) / 2
    intersects_all = []
    for i in range(1, n):
        for j in range(i):
            cros = np.cross(lines[:,i], lines[:,j])
            cros = cros / cros[2]
            intersects_all.append(cros)
    
    intersects_all = np.array(intersects_all)
    print(intersects_all)
    return np.mean(intersects_all, axis=0)

In [14]:
import numpy.linalg as la
def get_horizon_line(vanishing_points):
    """
    Calculates the ground horizon line.
    """
    horizon_line = np.cross(vanishing_points[:,0], vanishing_points[:,2])
    horizon_line = horizon_line / la.norm(horizon_line[:2])
    
    return horizon_line

In [8]:
def plot_horizon_line(im, horizon_line):
    """
    Plots the horizon line.
    """
    # <YOUR IMPLEMENTATION>
    plt.figure()
    plt.imshow(im)
    plt.show()
    #ax + by + c = 0
    #y=-c-ax / b
    a,b,c = horizon_line
    height, width, channel = im.shape
    x = np.array([0, width])
    y = (-1 * c - a * x) / b
    print(y)
    plt.plot(x,y)
    plt.show()

In [34]:
import numpy.linalg as la
def get_camera_parameters(vpts):
    """
    Computes the camera parameters. Hint: The SymPy package is suitable for this.
    """
    X = np.ones((3,3))
    vp0 = vpts[:-1,0]
    vp1 = vpts[:-1,1]
    vp2 = vpts[:-1,2]
    vp01_sum = vp0 + vp1
    vp02_sum = vp0 + vp2
    vp12_sum = vp1 + vp2
    vp01_product = vp0 * vp1
    vp02_product = vp0 * vp2
    vp12_product = vp1 * vp2

    X[:,0] = np.array([vp01_sum[0], vp02_sum[0], vp12_sum[0]])
    X[:,1] = np.array([vp01_sum[1], vp02_sum[1], vp12_sum[1]])
    rhs = np.array([-1*vp01_product[0] - vp01_product[1], -1*vp02_product[0] - vp02_product[1], -1*vp12_product[0] - vp12_product[1]])
    u,v,f = la.solve(X, rhs)
    u = -1 * u
    v = -1 * v
    f = (f - u**2 - v**2)**0.5
    return f, u, v

    pass

In [10]:
def get_rotation_matrix(f,u,v, vpts):
    """
    Computes the rotation matrix using the camera parameters.
    """
    #vpts = left, right, vertical
    K = np.array([[f,0,u],[0,f,v],[0,0,1]])
    R = la.inv(K) @ vpts
    norm = la.norm(R, axis=0)
    R[:,0] = R[:,0] / norm[0]
    R[:,1] = R[:,1] / norm[1]
    R[:,2] = R[:,2] / norm[2]
    return R

In [39]:
def estimate_height(reference_pos, vpts, pos, reference_height):
    """
    Estimates height for a specific object using the recorded coordinates. You might need to plot additional images here for
    your report.
    """
    v = np.cross(np.cross(pos[:,1], reference_pos[:,1]),np.cross(vpts[:,2], vpts[:,0]))
    v = v / v[2]
    t = np.cross(np.cross(v, reference_pos[:,0]), np.cross(pos[:,0], pos[:,1]))
    t = t / t[2]

    t_minus_b = la.norm(t - pos[:,1])
    r_minus_b = la.norm(pos[:,0] - pos[:,1])
    vvertical_minus_r = la.norm(vpts[:,1] - pos[:,0])
    vvertical_minus_t = la.norm(vpts[:,1] - t)
    height = reference_height * r_minus_b* vvertical_minus_t / (t_minus_b * vvertical_minus_r)
    print(height)
    return height
    
    
    pass

# Main function

In [27]:
im = np.asarray(Image.open('CSL.jpg'))

# Part 1
# Get vanishing points for each of the directions
num_vpts = 3
vpts = np.zeros((3, num_vpts))
print("input horizontal vanishing points first")
for i in range(num_vpts):
    print('Getting vanishing point %d' % i)
    # Get at least three lines from user input
    n, lines, centers = get_input_lines(im)
    # <YOUR IMPLEMENTATION> Solve for vanishing point
    vpts[:, i] = get_vanishing_point(lines)
    # Plot the lines and the vanishing point
    plot_lines_and_vp(im, lines, vpts[:, i])

# <YOUR IMPLEMENTATION> Get the ground horizon line
horizon_line = get_horizon_line(vpts)
# <YOUR IMPLEMENTATION> Plot the ground horizon line
plot_horizon_line(im, horizon_line)


input horizontal vanishing points first
Getting vanishing point 0
Set at least 3 lines to compute vanishing point
Click the two endpoints, use the right key to undo, and use the middle key to stop input
Click the two endpoints, use the right key to undo, and use the middle key to stop input
Click the two endpoints, use the right key to undo, and use the middle key to stop input
Click the two endpoints, use the right key to undo, and use the middle key to stop input
[[1.35871909e+03 2.27280441e+02 1.00000000e+00]
 [1.35579986e+03 2.26724398e+02 1.00000000e+00]
 [1.35401920e+03 2.27356246e+02 1.00000000e+00]]
Getting vanishing point 1
Set at least 3 lines to compute vanishing point
Click the two endpoints, use the right key to undo, and use the middle key to stop input
Click the two endpoints, use the right key to undo, and use the middle key to stop input
Click the two endpoints, use the right key to undo, and use the middle key to stop input
Click the two endpoints, use the right key t

In [32]:
horizon_line = get_horizon_line(vpts)
print(horizon_line[0]**2 , horizon_line[1]**2)
print(horizon_line)


2.029549331133293e-06 0.9999979704506688
[ 1.42462252e-03 -9.99998985e-01  2.25188088e+02]


In [44]:

# Part 2
# <YOUR IMPLEMENTATION> Solve for the camera parameters (f, u, v)
f, u, v = get_camera_parameters(vpts)
print(f, u, v)

# Part 3
# <YOUR IMPLEMENTATION> Solve for the rotation matrix
R = get_rotation_matrix(f,u,v,vpts)
print(R)
# Part 4
# Record image coordinates for each object and store in map
objects = ('person', 'CSL building', 'the spike statue', 'the lamp posts')
coords = dict()
reference_height = 1.68
for obj in objects:
    coords[obj] = get_top_and_bottom_coordinates(im, obj)

# <YOUR IMPLEMENTATION> Estimate heights
for obj in objects[1:]:
    print('Estimating height of %s' % obj)
    height = estimate_height(coords['person'], vpts, coords[obj], reference_height)

765.5233269011928 556.3212597477082 356.53979633140676
[[ 0.71755684 -0.00140434 -0.69649854]
 [-0.11610284  0.98576538 -0.12160075]
 [ 0.68675491  0.16812091  0.70717964]]
Click on the top coordinate of person
Click on the bottom coordinate of person
Click on the top coordinate of CSL building
Click on the bottom coordinate of CSL building
Click on the top coordinate of the spike statue
Click on the bottom coordinate of the spike statue
Click on the top coordinate of the lamp posts
Click on the bottom coordinate of the lamp posts
Estimating height of CSL building
81.86104709170603
Estimating height of the spike statue
30.934078167091275
Estimating height of the lamp posts
2.986133237121464


In [82]:
import scipy.linalg as sla
a = np.array([[2,0,3],[0,2,2],[0,0,1]])
print(a)
b = la.inv(a)
c = b.T @ b
print(c)
c = la.inv(c)
lbd, E = la.eig(c)
E

[[2 0 3]
 [0 2 2]
 [0 0 1]]
[[ 0.25  0.   -0.75]
 [ 0.    0.25 -0.5 ]
 [-0.75 -0.5   4.25]]


array([[ 8.13472176e-01,  5.54700196e-01, -1.74844821e-01],
       [ 5.42314784e-01, -8.32050294e-01, -1.16563214e-01],
       [ 2.10137322e-01, -7.71726946e-17,  9.77671880e-01]])